# SPM Tutorial #5: Statistics and Modeling

## Chapters
1. The Time-Series
2. History of the BOLD Signal
3. The Hemodynamic Response Function (HRF)
4. The General Linear Model
5. Creating Timing Files
6. Running the First-Level Analysis

## What is the GLM in fMRI?
The General Linear Model expresses the fMRI signal as:
$$Y = X\beta + \epsilon$$

Where:
- **Y**: Observed fMRI signal (time series for each voxel)
- **X**: Design matrix (regressors representing task and confounds)
- **β**: Parameter estimates (effect sizes for each regressor)
- **ε**: Error/noise term

In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
import shutil
import subprocess
from pathlib import Path

bids_root = Path("ds000102")
bold_file = Path("sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz")
full_bold_path = bids_root / bold_file
datalad = shutil.which("datalad")

if datalad and (not full_bold_path.exists() or full_bold_path.is_symlink()):
    print(f"Fetching BOLD data: {bold_file}")
    subprocess.run([datalad, "get", str(bold_file)], cwd=bids_root)

img = nib.load(full_bold_path)
data = img.get_fdata()
affine = img.affine
header = img.header
tr = header['pixdim'][4]

events_run1 = pd.read_csv("ds000102/sub-01/func/sub-01_task-flanker_run-1_events.tsv", sep="\t")

In [ ]:
from scipy.signal import convolve
import math

print("\n" + "=" * 70)
print("GENERAL LINEAR MODEL (GLM) SETUP")
print("=" * 70)

n_volumes = data.shape[3]
n_regressors = 0

print("\n1. HEMODYNAMIC RESPONSE FUNCTION (HRF)")
print("-" * 70)

def canonical_hrf(t, peak1=6, peak2=16, a1=6, a2=12, c=0.1):
    """Canonical HRF based on sum of two gamma functions"""
    g1 = (t**a1) * np.exp(-t/peak1) / (peak1**a1 * np.exp(-peak1) * math.factorial(a1-1))
    g2 = c * (t**a2) * np.exp(-t/peak2) / (peak2**a2 * np.exp(-peak2) * math.factorial(a2-1))
    return g1 - g2

hrf_time = np.arange(0, 30, tr)
hrf = canonical_hrf(hrf_time)
hrf = hrf / np.max(hrf)

print(f"HRF parameters:")
print(f"  Time to peak: ~6 seconds")
print(f"  Post-undershoot peak: ~16 seconds")
print(f"  Sampling rate: {1/tr:.2f} Hz (TR = {tr:.2f}s)")
print(f"  HRF samples: {len(hrf)}")

print("\n2. TASK REGRESSORS")
print("-" * 70)

congruent_onsets = events_run1[events_run1['Stimulus'] == 'congruent']['onset'].values
congruent_regressor = np.zeros(n_volumes)
for onset in congruent_onsets:
    idx = int(onset / tr)
    if idx < n_volumes:
        congruent_regressor[idx] = 1

print(f"Congruent trials: {len(congruent_onsets)}")
print(f"  Onsets: {congruent_onsets[:3]}... (seconds)")

incongruent_onsets = events_run1[events_run1['Stimulus'] == 'incongruent']['onset'].values
incongruent_regressor = np.zeros(n_volumes)
for onset in incongruent_onsets:
    idx = int(onset / tr)
    if idx < n_volumes:
        incongruent_regressor[idx] = 1

print(f"Incongruent trials: {len(incongruent_onsets)}")
print(f"  Onsets: {incongruent_onsets[:3]}... (seconds)")

congruent_hrf = convolve(congruent_regressor, hrf)[:len(congruent_regressor)]
incongruent_hrf = convolve(incongruent_regressor, hrf)[:len(incongruent_regressor)]

n_regressors = 2
print(f"\nRegressors after HRF convolution: {n_regressors}")

print("\n3. CONFOUND REGRESSORS")
print("-" * 70)

motion_regressors = np.zeros((n_volumes, 6))
motion_derivatives = np.diff(motion_regressors, axis=0, prepend=0)
drift = np.polyfit(np.arange(n_volumes), np.arange(n_volumes), 1)
linear_drift = np.polyval(drift, np.arange(n_volumes))

n_confounds = 6 + 6 + 1
print(f"Confound regressors: {n_confounds}")
print(f"  Motion parameters: 6 (translation + rotation)")
print(f"  Motion derivatives: 6")
print(f"  Linear drift: 1")

print("\n4. DESIGN MATRIX")
print("-" * 70)

X = np.column_stack([
    congruent_hrf,
    incongruent_hrf,
    motion_regressors,
    motion_derivatives,
    linear_drift
])
print(f"Design matrix shape: {X.shape}")
print(f"  Time points (rows): {X.shape[0]}")
print(f"  Regressors (columns): {X.shape[1]}")
print(f"    - Task regressors: 2")
print(f"    - Motion + derivatives: 12")
print(f"    - Drift: 1")

cond_num = np.linalg.cond(X.T @ X)
print(f"\nDesign matrix conditioning: {cond_num:.2e}")
if cond_num < 1e10:
    print("  ✓ Well-conditioned matrix")
else:
    print("  ! High condition number (multicollinearity)")

print("\n✓ GLM Setup Complete")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## GLM Visualizations - Design Matrix and HRF
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('General Linear Model: Design Matrix Components', fontsize=14, fontweight='bold')

ax = axes[0, 0]
ax.plot(hrf_time, hrf, 'b-', linewidth=2.5)
ax.fill_between(hrf_time, hrf, alpha=0.3)
ax.axhline(0, color='k', linestyle='--', alpha=0.3)
ax.set_xlabel('Time (seconds)', fontweight='bold')
ax.set_ylabel('HRF Amplitude', fontweight='bold')
ax.set_title('Canonical Hemodynamic Response Function', fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[0, 1]
n_show = min(50, n_volumes)
time_axis = np.arange(n_show) * tr
ax.plot(time_axis, congruent_hrf[:n_show], 'g-', linewidth=2, label='Congruent (HRF)', alpha=0.8)
ax.plot(time_axis, incongruent_hrf[:n_show], 'r-', linewidth=2, label='Incongruent (HRF)', alpha=0.8)
for onset in congruent_onsets:
    if onset < time_axis[-1]:
        ax.axvline(onset, color='green', alpha=0.2, linewidth=3)
for onset in incongruent_onsets:
    if onset < time_axis[-1]:
        ax.axvline(onset, color='red', alpha=0.2, linewidth=3)
ax.set_xlabel('Time (seconds)', fontweight='bold')
ax.set_ylabel('HRF Amplitude', fontweight='bold')
ax.set_title('Task Regressors (First 50 TRs)', fontweight='bold')
ax.legend(loc='upper right')
ax.grid(alpha=0.3)

ax = axes[1, 0]
im = ax.imshow(X[:n_show, :10].T, aspect='auto', cmap='RdBu_r', vmin=-2, vmax=2)
ax.set_xlabel('Time Point (TR)', fontweight='bold')
ax.set_ylabel('Regressor', fontweight='bold')
ax.set_title('Design Matrix (First 50 TRs, 10 regressors)', fontweight='bold')
regressor_names = ['Congruent', 'Incongruent'] + [f'Motion{i+1}' for i in range(6)] + ['Drift', 'Drift2']
ax.set_yticks(range(10))
ax.set_yticklabels(regressor_names, fontsize=9)
plt.colorbar(im, ax=ax, label='Standardized Value')

ax = axes[1, 1]
X_std = (X - X.mean(axis=0)) / X.std(axis=0)
corr_matrix = np.corrcoef(X_std.T)
corr_subset = corr_matrix[:10, :10]
im = ax.imshow(corr_subset, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_xlabel('Regressor', fontweight='bold')
ax.set_ylabel('Regressor', fontweight='bold')
ax.set_title('Regressor Correlation Matrix (Subset)', fontweight='bold')
ax.set_xticks(range(10))
ax.set_xticklabels(regressor_names, rotation=45, ha='right', fontsize=9)
ax.set_yticks(range(10))
ax.set_yticklabels(regressor_names, fontsize=9)
plt.colorbar(im, ax=ax, label='Correlation')

plt.tight_layout()
plt.savefig('glm_design_matrix.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ GLM Design Matrix Visualization Complete")

In [ ]:
## GLM Estimation on Sample Voxel
print("\n" + "=" * 70)
print("GLM ESTIMATION ON SAMPLE VOXEL")
print("=" * 70)

mid_x, mid_y, mid_z = [d // 2 for d in data.shape[:3]]
sample_voxel = data[mid_x, mid_y, mid_z, :]
print(f"\nSample voxel signal shape: {sample_voxel.shape}")
print(f"  Mean intensity: {sample_voxel.mean():.2f}")
print(f"  Std dev: {sample_voxel.std():.2f}")

y = (sample_voxel - sample_voxel.mean()) / sample_voxel.std()
X_mean = X.mean(axis=0, keepdims=True)
X_std = X.std(axis=0, keepdims=True)
X_std[X_std == 0] = 1
X_norm = (X - X_mean) / X_std

XtX = X_norm.T @ X_norm
Xty = X_norm.T @ y
ridge_lambda = 1e-6
beta = np.linalg.solve(XtX + ridge_lambda * np.eye(XtX.shape[0]), Xty)

y_pred = X_norm @ beta
residuals = y - y_pred

mse = np.mean(residuals**2)
sse = np.sum(residuals**2)
tss = np.sum(y**2)
r_squared = 1 - (sse / tss)

residual_variance = np.sum(residuals**2) / (len(y) - X.shape[1])
cov_matrix = residual_variance * np.linalg.inv(XtX + ridge_lambda * np.eye(XtX.shape[0]))
se = np.sqrt(np.diag(cov_matrix))

t_stats = beta / se
dof = len(y) - X.shape[1]

print(f"\nEstimation Results:")
print(f"  R² = {r_squared:.4f}")
print(f"  MSE = {mse:.4f}")
print(f"  Residual variance = {residual_variance:.4f}")
print(f"  Degrees of freedom = {dof}")

print(f"\nParameter Estimates (Task Regressors):")
print(f"  Congruent effect:")
print(f"    β = {beta[0]:.4f}")
print(f"    SE = {se[0]:.4f}")
print(f"    t({dof}) = {t_stats[0]:.4f}")
print(f"  Incongruent effect:")
print(f"    β = {beta[1]:.4f}")
print(f"    SE = {se[1]:.4f}")
print(f"    t({dof}) = {t_stats[1]:.4f}")

contrast_vector = np.array([-1, 1] + [0] * (X.shape[1] - 2))
contrast_beta = contrast_vector @ beta
contrast_var = contrast_vector @ cov_matrix @ contrast_vector
contrast_se = np.sqrt(contrast_var)
contrast_t = contrast_beta / contrast_se

print(f"\nContrast: Incongruent > Congruent")
print(f"  β_contrast = {contrast_beta:.4f}")
print(f"  SE = {contrast_se:.4f}")
print(f"  t({dof}) = {contrast_t:.4f}")

print("\n✓ GLM Estimation Complete")

In [ ]:
## GLM Results Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('GLM Results: Voxel-Level Analysis', fontsize=14, fontweight='bold')

ax = axes[0, 0]
time_points = np.arange(len(y)) * tr
ax.plot(time_points, y, 'k.', alpha=0.5, markersize=3, label='Observed signal')
ax.plot(time_points, y_pred, 'r-', linewidth=2, label='Model fit')
ax.fill_between(time_points, y_pred - np.sqrt(mse), y_pred + np.sqrt(mse),
                 alpha=0.2, color='red', label='±1 SD error')
ax.set_xlabel('Time (seconds)', fontweight='bold')
ax.set_ylabel('Standardized Signal', fontweight='bold')
ax.set_title('Time Series and GLM Fit', fontweight='bold')
ax.legend(loc='upper right')
ax.grid(alpha=0.3)

ax = axes[0, 1]
ax.plot(time_points, residuals, 'purple', alpha=0.6, linewidth=1)
ax.axhline(0, color='k', linestyle='--', alpha=0.3)
ax.set_xlabel('Time (seconds)', fontweight='bold')
ax.set_ylabel('Residual Value', fontweight='bold')
ax.set_title('Model Residuals', fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[1, 0]
param_names = ['Cong', 'Incong'] + [f'M{i}' for i in range(6)]
beta_subset = beta[:len(param_names)]
se_subset = se[:len(param_names)]
y_pos = np.arange(len(param_names))
ax.barh(y_pos, beta_subset, xerr=1.96*se_subset, capsize=5, alpha=0.7, color='steelblue', edgecolor='black')
ax.axvline(0, color='k', linestyle='-', linewidth=0.5)
ax.set_yticks(y_pos)
ax.set_yticklabels(param_names)
ax.set_xlabel('Parameter Estimate', fontweight='bold')
ax.set_title('Parameter Estimates (95% CI)', fontweight='bold')
ax.grid(alpha=0.3, axis='x')

ax = axes[1, 1]
from scipy import stats
stats.probplot(residuals, dist="norm", plot=ax)
ax.set_title('Q-Q Plot: Residual Normality', fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('glm_results.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ GLM Results Visualization Complete")